In [122]:
import psycopg2
import os
import gc
import geoalchemy2
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine

In [ ]:
Type: PostgreSQL
Host: 15.236.82.244
Port: 5432
Database: mapineq
User: postgres
Password: Fni7fUqnbXzEURcf

In [110]:
user = 'postgres'
password = 'Fni7fUqnbXzEURcf'
host = '15.236.82.244'
database_name = 'mapineq'
engine = create_engine('postgresql://%s:%s@%s/%s'%(user,password,host,database_name))

In [125]:
column_dict = {'NUTS_ID':'id', 'CNTR_CODE':'country', 'LEVL_CODE':'level', 'NUTS_NAME':'name', 'geometry':'geometry'}
nuts.rename(columns=column_dict, inplace=True)

In [126]:
nuts

,id,level,country,name,FID,geometry
0,NL11,2,NL,Groningen,NL11,"MULTIPOLYGON (((4107504.262 3333865.328, 41051..."
1,IE01,2,IE,"Border, Midland and Western",IE01,"MULTIPOLYGON (((3065716.058 3502970.616, 30658..."
2,NL12,2,NL,Friesland,NL12,"MULTIPOLYGON (((4004978.096 3338216.912, 40048..."
3,DE93,2,DE,Lüneburg,DE93,"POLYGON ((4428069.786 3326489.110, 4428174.658..."
4,NL13,2,NL,Drenthe,NL13,"POLYGON ((4118354.565 3285295.884, 4118157.220..."
...,...,...,...,...,...,...
1886,HR0,1,HR,Hrvatska,HR0,"MULTIPOLYGON (((4827370.400 2618319.743, 48274..."
1887,RO1,1,RO,Macroregiunea unu,RO1,"POLYGON ((5322772.075 2853513.182, 5323386.908..."
1888,RO2,1,RO,Macroregiunea doi,RO2,"MULTIPOLYGON (((5853872.205 2619332.314, 58537..."
1889,RO3,1,RO,Macroregiunea trei,RO3,"POLYGON ((5513343.726 2597553.157, 5513421.210..."


In [127]:
nuts[column_dict.values()].to_postgis('nuts2003', engine, schema='public', if_exists='replace')

In [ ]:
# get ready for GADM 
gadm = gpd.read_file('/data/xiang/1-Data/GADM/gadm_410-gpkg/gadm_410.gpkg')

def get_unique(piece):
    unique = ''
    id = 5
    while unique == '':
        unique  = piece['GID_'+str(id)]
        id-=1
    return unique

gadm.to_file('/data/xiang/1-Data/GADM/gadm_410-gpkg/gadm.gpkg')

In [29]:
# get ready for nuts
nuts_folder = '/data/xiang/1-Data/NUTS'
save_folder = '/data/xiang/1-Data/NUTS/nuts'
for file in os.listdir(nuts_folder):
    if file.endswith('.shp') and file.startswith('NUTS'):
        nuts = gpd.read_file(nuts_folder + '//' + file)
        nuts.rename(columns={'NUTS_ID':'geo'})
        nuts.to_file(save_folder + '//' + file)

In [124]:
nuts_path = '/data/xiang/1-Data/NUTS/NUTS_RG_01M_2003_3035.shp'
nuts = gpd.read_file(nuts_path)
nuts

,NUTS_ID,LEVL_CODE,CNTR_CODE,NUTS_NAME,FID,geometry
0,NL11,2,NL,Groningen,NL11,"MULTIPOLYGON (((4107504.262 3333865.328, 41051..."
1,IE01,2,IE,"Border, Midland and Western",IE01,"MULTIPOLYGON (((3065716.058 3502970.616, 30658..."
2,NL12,2,NL,Friesland,NL12,"MULTIPOLYGON (((4004978.096 3338216.912, 40048..."
3,DE93,2,DE,Lüneburg,DE93,"POLYGON ((4428069.786 3326489.110, 4428174.658..."
4,NL13,2,NL,Drenthe,NL13,"POLYGON ((4118354.565 3285295.884, 4118157.220..."
...,...,...,...,...,...,...
1886,HR0,1,HR,Hrvatska,HR0,"MULTIPOLYGON (((4827370.400 2618319.743, 48274..."
1887,RO1,1,RO,Macroregiunea unu,RO1,"POLYGON ((5322772.075 2853513.182, 5323386.908..."
1888,RO2,1,RO,Macroregiunea doi,RO2,"MULTIPOLYGON (((5853872.205 2619332.314, 58537..."
1889,RO3,1,RO,Macroregiunea trei,RO3,"POLYGON ((5513343.726 2597553.157, 5513421.210..."


In [107]:
nuts['difference'] = nuts.apply(lambda x: True if x['NUTS_ID'] != x['FID'] else False, axis=1)

In [108]:
nuts[nuts['difference']==True]

,NUTS_ID,LEVL_CODE,CNTR_CODE,NUTS_NAME,FID,geometry,char,difference


In [109]:
nuts

,NUTS_ID,LEVL_CODE,CNTR_CODE,NUTS_NAME,FID,geometry,char,difference
0,NL11,2,NL,Groningen,NL11,"MULTIPOLYGON (((4107504.262 3333865.328, 41051...",4,False
1,IE01,2,IE,"Border, Midland and Western",IE01,"MULTIPOLYGON (((3065716.058 3502970.616, 30658...",4,False
2,NL12,2,NL,Friesland,NL12,"MULTIPOLYGON (((4004978.096 3338216.912, 40048...",4,False
3,DE93,2,DE,Lüneburg,DE93,"POLYGON ((4428069.786 3326489.110, 4428174.658...",4,False
4,NL13,2,NL,Drenthe,NL13,"POLYGON ((4118354.565 3285295.884, 4118157.220...",4,False
...,...,...,...,...,...,...,...,...
1886,HR0,1,HR,Hrvatska,HR0,"MULTIPOLYGON (((4827370.400 2618319.743, 48274...",3,False
1887,RO1,1,RO,Macroregiunea unu,RO1,"POLYGON ((5322772.075 2853513.182, 5323386.908...",3,False
1888,RO2,1,RO,Macroregiunea doi,RO2,"MULTIPOLYGON (((5853872.205 2619332.314, 58537...",3,False
1889,RO3,1,RO,Macroregiunea trei,RO3,"POLYGON ((5513343.726 2597553.157, 5513421.210...",3,False


In [105]:
nuts.sort_values('char', ascending=False)

,NUTS_ID,LEVL_CODE,CNTR_CODE,NUTS_NAME,FID,geometry,char
876,DE711,3,DE,"Darmstadt, Kreisfreie Stadt",DE711,"POLYGON ((4229544.832 2983152.227, 4229963.220...",5
1131,DE148,3,DE,Ravensburg,DE148,"POLYGON ((4330888.525 2745176.256, 4331051.819...",5
1141,FR241,3,FR,Cher,FR241,"POLYGON ((3719568.984 2621833.751, 3719378.303...",5
1140,DEB3H,3,DE,Südliche Weinstraße,DEB3H,"POLYGON ((4166207.657 2910128.277, 4166370.461...",5
1139,DEB38,3,DE,"Speyer, Kreisfreie Stadt",DEB38,"POLYGON ((4209451.936 2908945.397, 4207654.314...",5
...,...,...,...,...,...,...,...
1163,CH,0,CH,Schweiz/Suisse/Svizzera,CH,"POLYGON ((4018239.470 2597253.828, 4019057.890...",2
898,NL,0,NL,Nederland,NL,"MULTIPOLYGON (((3923159.514 3156174.530, 39206...",2
891,PT,0,PT,Portugal,PT,"MULTIPOLYGON (((2777973.894 1743031.454, 27780...",2
890,TR,0,TR,Turkiye,TR,"MULTIPOLYGON (((5836705.361 1816063.955, 58367...",2


In [102]:
nuts_path = '/data/xiang/1-Data/NUTS/NUTS_RG_01M_2021_3035.shp'
nuts_21 = gpd.read_file(nuts_path)
nuts_21

,NUTS_ID,LEVL_CODE,CNTR_CODE,NAME_LATN,NUTS_NAME,MOUNT_TYPE,URBN_TYPE,COAST_TYPE,FID,geometry
0,AL,0,AL,Shqipëria,Shqipëria,0.0,0,0,AL,"MULTIPOLYGON (((5129579.170 2204098.752, 51298..."
1,CZ,0,CZ,Česko,Česko,0.0,0,0,CZ,"POLYGON ((4635755.041 3113254.655, 4636313.459..."
2,DE,0,DE,Deutschland,Deutschland,0.0,0,0,DE,"MULTIPOLYGON (((4355225.365 2715902.993, 43541..."
3,DK,0,DK,Danmark,Danmark,0.0,0,0,DK,"MULTIPOLYGON (((4650502.736 3591342.844, 46503..."
4,CY,0,CY,Kýpros,Κύπρος,0.0,0,0,CY,"MULTIPOLYGON (((6527040.718 1762367.593, 65267..."
...,...,...,...,...,...,...,...,...,...,...
2005,NO0B1,3,NO,Jan Mayen,Jan Mayen,3.0,3,1,NO0B1,"POLYGON ((3673081.665 5420781.907, 3673257.365..."
2006,EE009,3,EE,Kesk-Eesti,Kesk-Eesti,4.0,3,1,EE009,"MULTIPOLYGON (((5274914.971 4151899.845, 52742..."
2007,NO0,1,NO,Norge,Norge,0.0,0,0,NO0,"MULTIPOLYGON (((5073773.420 5207018.495, 50700..."
2008,NO0B,2,NO,Jan Mayen and Svalbard,Jan Mayen and Svalbard,NaN,0,0,NO0B,"MULTIPOLYGON (((4760536.161 6404487.632, 47607..."


In [57]:
# get ready for urban green
UA_folder = '/data/xiang/3-case studies/1-urban atlas/results'
UA_comb = None
for file in os.listdir(UA_folder):
    UA = pd.read_csv(UA_folder + '//' + file, index_col=0)
    year = file.split('.')[0].split(' ')[2][-4:]
    UA['geo_source'] = 'NUTS_' + year
    UA_comb = UA if UA_comb is None else pd.concat([UA_comb,UA])
UA_comb.reset_index(inplace=True)
UA_comb.drop('index', axis=1, inplace=True)
UA_comb.to_csv('/data/xiang/3-case studies/1-urban atlas/results/UA_nuts.csv')

In [64]:
# get ready for ookla
ookla_folder = '/data/xiang/3-case studies/0-ookla data'
ookla_comb = None
for file in os.listdir(ookla_folder):
    if file.endswith('.csv'):
        ookla = pd.read_csv(ookla_folder + '//' + file, index_col=0)
        year = file.split('.')[0].split('_')[1][-4:]
        ookla['geo_source'] = 'NUTS_' + year
        ookla_comb = ookla if ookla_comb is None else pd.concat([ookla_comb,ookla])
ookla_comb.reset_index(inplace=True)
ookla_comb.drop('index', axis=1, inplace=True)
ookla_comb.to_csv(ookla_folder + '//' + 'ookla_nuts.csv')

In [79]:
# get ready for pm
pm_folder = '/data/xiang/3-case studies/2-PM2.5'
pm_comb = None
for file in os.listdir(pm_folder):
    if file.endswith('.csv'):
        pm = pd.read_csv(pm_folder + '//' + file)
        year = file.split('.')[0].split('_')[1][-4:]
        pm['geo_source'] = 'NUTS_' + year
        pm_comb = pm if pm_comb is None else pd.concat([pm_comb,pm])
pm_comb.reset_index(inplace=True)
pm_comb.drop('index', axis=1, inplace=True)
pm_comb.to_csv(pm_folder + '//' + 'pm_nuts.csv')

In [96]:
# get ready for smod for nuts
smod_folder = '/data/xiang/3-case studies/3-GHS_SMOD'
indicator_list = ['URBAN CENTRE GRID CELL','DENSE URBAN CLUSTER GRID CELL','SEMI-DENSE URBAN CLUSTER GRID CELL',
                  'SUBURBAN OR PERI-URBAN GRID CELL','RURAL CLUSTER GRID CELL','LOW DENSITY RURAL GRID CELL',
                  'VERY LOW DENSITY RURAL GRID CELL','WATER GRID CELL']
for indicator in indicator_list:
    smod_comb = None
    for file in os.listdir(smod_folder):
        if file.split('.')[0].split('_')[-1] == indicator:
            smod = pd.read_csv(smod_folder + '//' + file)
            year = file.split('.')[0].split('_')[2][-4:]
            smod['geo_source'] = 'NUTS_' + year
            smod_comb = smod if smod_comb is None else pd.concat([smod_comb, smod])
    smod_comb.reset_index(inplace=True)
    smod_comb.drop('index', axis=1, inplace=True)
    smod_comb.to_csv('/data/xiang/3-case studies/3-GHS_SMOD/nuts_combine' + '//' + 'SMOD_' + indicator + '.csv')

In [ ]:
# upload smod
'DROP TABLE IF EXISTS'+SMOD_NUTS+'CASCADE;\n'+'CREATE TABLE'+SMOD_NUTS+
'(\ngeo_source	TEXT,\ngeo			TEXT,\nindicator	TEXT,\nfreq		TEXT,\nunit		TEXT,\n"obsTime"	TEXT,\n"obsValue"	TEXT);'+
'truncate table'+SMOD_NUTS+'\\'+'copy'+SMOD_NUTS+'FROM'
+SMOD_path +'HEADER DELIMITER'+"','"+'CSV'+
'INSERT INTO public.catalogue(\n'+'provider, resource, descr,   use, short_descr)\n'+
	"VALUES ('Global Human Settlement',"+SMOD_NUTS+', 'SMOD_NUTS+', '+'TRUE, '+SMOD_NUTS+');'
+
	
'CREATE OR REPLACE VIEW vw_'+ SMOD_path AS
SELECT
	geo,
	indicator,
	freq,
	unit,
	"obsTime",
	"obsValue"
FROM
	pm25;

In [100]:
print('CASCADE;\nCREATE TABLE SMOD_NUTS/n(/ngeo_source	TEXT,/ngeo			TEXT,/nindicator	TEXT,/nfreq		TEXT,/n')

CASCADE;
CREATE TABLE SMOD_NUTS/n(/ngeo_source	TEXT,/ngeo			TEXT,/nindicator	TEXT,/nfreq		TEXT,/n
